<a href="https://colab.research.google.com/github/WuraolaOyewusi/How-to-use-scispaCy-Entity-Linkers-for-Biomedical-Named-Entities/blob/master/scispacy_entities_extractions_and_linkers(cleared_outputs)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Download extracted entities dataset, run this cell if you want to go straight to entity linkage without extracting the Named Entities yourself.If you do this, you do not need to install the NER models and import them.

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/WuraolaOyewusi/How-to-use-scispaCy-Entity-Linkers-for-Biomedical-Named-Entities/master/entities_and_label_from_4_scispacy_NER_models.csv

In [ ]:
%%capture                                                               
!pip install scispacy
!pip install swifter
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz      #scispacy medium model
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz      #scispacy medium model
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_bc5cdr_md-0.2.5.tar.gz    #biomedical NER model trained on BC5CDR corpus
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_bionlp13cg_md-0.2.5.tar.gz  #biomedical NER model trained on BIONLP13CG corpus
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_craft_md-0.2.5.tar.gz    #biomedical NER model trained on CRAFT corpus
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_jnlpba_md-0.2.5.tar.gz     #biomedical NER model trained on JNLPBA corpus

In [ ]:
import spacy
import scispacy
import swifter
import pandas as pd
from spacy import displacy
import en_core_sci_sm
#import en_core_sci_md
# import en_ner_bc5cdr_md
# import en_ner_jnlpba_md
# import en_ner_craft_md
# import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter
from pprint import pprint
from tqdm import tqdm
tqdm.pandas()

### The sample text here is from 
###Denison MR. CORONAVIRUS RESEARCH: KEYS TO DIAGNOSIS, TREATMENT, AND PREVENTION OF SARS. In: Institute of Medicine (US) Forum on Microbial Threats; Knobler S, Mahmoud A, Lemon S, et al., editors. Learning from SARS: Preparing for the Next Disease Outbreak: Workshop Summary. Washington (DC): National Academies Press (US); 2004. Available from: https://www.ncbi.nlm.nih.gov/books/NBK92477/

In [ ]:
sample_text= """CORONAVIRUS RESEARCH: KEYS TO DIAGNOSIS, TREATMENT, AND PREVENTION OF SARS
Mark R. Denison, M.D.
For coronavirus investigators, the recognition of a new coronavirus as the cause of severe acute respiratory syndrome (SARS) was certainly remarkable, yet perhaps not surprising (Baric et al., 1995). The cadre of investigators who have worked with this intriguing family of viruses over the past 30 years are familiar with many of the features of coronavirus biology, pathogenesis, and disease that manifested so dramatically in the worldwide SARS epidemic. Advances in the biology of coronaviruses have resulted in greater understanding of their capacity for adaptation to new environments, transspecies infection, and emergence of new diseases. New tools of cell and molecular biology have led to increased understanding of intracellular replication and viral cell biology, and the advent in the past five years of reverse genetic approaches to study coronaviruses has made it possible to begin to define the determinants of viral replication, transpecies adaptation, and human disease. This summary will discuss the basic life cycle and replication of the well-studied coronavirus, mouse hepatitis virus (MHV), identifying the unique characteristics of coronavirus biology and highlighting critical points where research has made significant advances, and which might represent targets for antivirals or vaccines. Areas where rapid progress has been made in SCoV research will be described. Finally, areas of need for research in coronavirus replication, genetics, and pathogenesis will be summarized.
Coronavirus Life Cycle
The best studied model for coronavirus replication and pathogenesis has been the group 2 murine coronavirus, mouse hepatitis virus, and much of what is known of the stages of the coronavirus life cycle has been determined in animals and in culture using this virus. Thus this discussion will focus on MHV with comparisons to SCoV and other coronaviruses. This is appropriate because bioinformatics analyses suggest that SCoV, while a distinct virus, has significant similarities in organization, putative protein functions, and replication to the group II coronaviruses, particularly within the replicase gene (Snijder et al., 2003). Excellent, detailed reviews of MHV and coronavirus replication are available elsewhere (Holmes and Lai, 1996; Lai and Cavanagh, 1997).
The coronavirus virion is an enveloped particle containing the spike (S), membrane (M), and envelope (E) proteins. In addition, some strains of coronaviruses, but not SCoV, express a hemagglutinin protein (HE) that is also incorporated in the virion. The genome of coronaviruses is a linear, single-stranded RNA molecule of positive (mRNA) polarity, and from 28 to 32 kb in length (Bonilla et al., 1994; Drosten et al., 2003; Lee et al., 1991). Within the virion, the genome is encapsidated by multiple copies of the nucleocapsid protein (N), and has the conformation of a helical RNA/nucleocapsid structure. The S protein has been a focus of pathogenesis studies in mice because it appears to be the critical determinant of cell tropism, species specificity, host selection, cell tropism, and disease (Navas and Weiss, 2003; Navas et al., 2001; Rao and Gallagher, 1998).
Virus replication is initiated by binding of the S protein to specific receptors on the host cell surface. For MHV, the primary receptor has been shown to be the carcino-embryonic antigen–cell adhesion molecule (CEACAM) (Dveksler et al., 1991; 1996; Holmes and Lai, 1996; Yokomuri and Lai, 1992), and for the human coronavirus, HCoV-229E, and other group 1 coronaviruses, the receptor is aminopeptidase N (Yeager et al., 1992). The precise mechanisms of entry and uncoating have yet to be defined, but likely occur by either fusion from without or viroplexis through endocytic vesicles. For wildtype MHV, entry and uncoating constitute a pH independent process that is probably direct fusion mediated by a fusion peptide in the S protein (Gallagher et al., 1991). The understanding of the region of the S1 component of coronavirus that binds to receptors was the basis for studies leading to the very recent and very rapid identification of angiotensin converting enzyme 2 (ACE 2) as a receptor for SCoV (Li et al., 2003).
The next discrete stage in the life cycle is translation and proteolytic processing of viral replicase proteins from the input genome RNA, followed by formation of cytoplasmic replication complexes in association with cellular membranes (Denison et al., 1999; Gosert et al., 2002; Shi et al., 1999; van der Meer et al., 1999). Replication complexes are thought to be sites of all stages of viral RNA transcription and replication, and possibly assembly of nascent viral nucleocapsids. Viral assembly occurs both temporally and physically distinct from viral replication complexes in the endoplasmic-reticulum-Golgi-intermediate compartment (ERGIC), a transitional zone between late ER and Golgi (deVries et al., 1997; Klumperman et al., 1994; Krijnse-Locker et al., 1994; Rottier and Rose, 1987). Although the mechanisms by which replication products are delivered to sites of assembly remain to be determined, it has been shown that subpopulations of replicase proteins and the structural nucleocapsid (N) translocate from replication complexes to sites of assembly and may mediate the process in association with cellular membrane/protein trafficking pathways (Bost et al., 2000). Virus assembly in the ERGIC involves interactions of genome RNA, N, the membrane protein (M), and the small membrane protein (E), resulting in budding of virions into the lumen of ER/Golgi virosomes (Opstetten et al., 1995). Further maturation of virus particles occurs during movement through the Golgi, resulting in virosomes filled with mature particles (Salamuera et al., 1999). Trafficking of the virosomes to the cell surface has not been well characterized, but is presumed to occur via normal vesicle maturation and exocytic processes. The outcome is the nonlytic release of the vast majority of mature virions into the extracellular space. For MHV and several other coronaviruses that can directly fuse with cells, there is a characteristic and rapidly detectable cytopathic effect of cell-cell fusion into multinucleated syncytia. Production of infectious virus continues even after the majority of cells are fused. Syncytia were recently reported as a readout of SCoV receptor expression and cell infection (Li et al., 2003).
Viral Replication Complex Formation and Function
Following entry and uncoating, the 5′ most replicase gene of the input positive strand RNA genome is translated into two co-amino terminal replicase polyproteins that are co- and post-translationally processed by viral proteinases to yield 15 to 16 mature replicase proteins, as well as intermediate precursors. The nascent replicase polyproteins and intermediate precursors likely mediate the formation of viral replication complexes in the host cell cytoplasm. Interestingly, coronavirus replication requires continuous replicase gene translation and processing throughout the life cycle to maintain productive infection (Kim et al., 1995; Perlman et al., 1987; Sawicki and Sawicki, 1986). Replication complexes of MHV are associated with double-membrane vesicles (Gosert et al., 2002), and all tested MHV replicase proteins have been shown to colocalize to replication complexes at the earliest time of detection, likely both by membrane integration and by protein-protein and protein-RNA interactions (Bost et al., 2000; Denison et al., 1999; Prentice and Denison, in press; Shi et al., 1999; Sims et al., 2000; van der Meer et al., 1999). Further, replicase proteins likely mediate the process of double-membrane vesicle formation, likely by induction of cellular autophagy pathways (E. Prentice, unpublished results).
Coronavirus replication complexes are sites for replicase gene translation and replicase polyprotein processing, and also for viral RNA synthesis. Replicase gene proteins likely mediate positive-strand, negative-strand, subgenomic, and genomic RNA synthesis, as well as processes of capping, polyadenylation, RNA unwinding, template switching during viral RNA synthesis, and discontinuous transcription and transcription attenuation. The coronavirus replicase polyproteins and mature replicase proteins represent the largest and most diverse repertoire of known and predicted distinct enzymatic functions of any positive-strand RNA virus family. Until recently, of the 15 or more mature replicase proteins, only the proteinase, RNA helicase, and RNA-dependent RNA polymerase activities had been predicted or experimentally confirmed (Brockway et al., 2003; Heusipp et al., 1997; Lee et al., 1991; Ziebuhr et al., 2000). With the advent of SARS, more extensive bioinformatics analyses have resulted in predictions of several additional functions involved in RNA processing, including methyltransferase and exonuclease activities (Snijder et al., 2003; Thiel et al., 2003). Even with inclusion of distant predicted relationships, up to eight of the replicase proteins remain without predicted or confirmed functions. In summary, it is likely that coronaviruses have exploited their genetic capacity to encode proteins in the replicase gene with distinct functions in RNA synthesis and processing, as well as proteins with specific roles in induction or modification in host cellular membrane biogenesis and trafficking, delivery of replication products to sites of assembly, and possibly virus assembly. Thus replicase translation, replicase polyprotein processing, and mature replicase proteins constitute important targets for interference with coronavirus replication, virus-cell interactions, or viral pathology.
Coronavirus Replicase Protein Expression and Processing
The proteinase activities for all coronaviruses include both papain-like proteinase (PLP) and picornavirus 3C-like proteinase activities that are encoded within the replicase polyproteins and mediate both cis and trans cleavage events (Ziebuhr et al., 2000). Because of the parallel evolution of the proteinases, their cleavage sites, and the hierarchical cleavage processes, the proteolytic processing of the coronavirus replicase proteins may serve as distinct regulatory and genetic elements (Ziebuhr et al., 2001). Specifically, there are both conserved and divergent regions of the replicase polyproteins by amino acid identity and similarity, with the sequences and predicted mature proteins beginning with the 3C-like proteinases through the carboxy terminus of the replicase polyprotein retaining higher identity and similarity across the predicted proteins. In contrast, the amino-terminal third of the replicase demonstrates the most variation in proteins, cleavage site locations, and the number of proteinases that mediate maturation processing. SCoV appears to have the general organization of, and similar protein sizes to, the group 2 coronaviruses such as MHV in this part of the genome (Snijder et al., 2003). However, SCoV likely uses only one PLP to mediate the cleavages, similar to the group 3 coronavirus infectious bronchitis virus (IBV). Thus this region of the replicase may experience the most variability, suggesting either the encoding of accessory functions that are flexible and tolerant of changes, or conversely group or host-specific roles that are subject to pressure for more rapid change.
Expression of Structural and Accessory Genes
Only the 5′ most replicase gene is translated from the input positive-strand genome RNA. The genome contains multiple other genes for the known structural proteins S, E, M, and N, as well as other genes for expression of proteins that have been labeled as “nonstructural” or “accessory” because they have been presumed to not be required for replication, and are not thought to be incorporated into virions. MHV encodes six of these genes, while SCoV encodes possibly up to 11 structural and accessory genes, which are expressed from subgenomic mRNAs (Snijder et al., 2003). Subgenomic RNA transcription occurs during minus- strand RNA synthesis by acquisition of the antileader RNA sequences from the 5′ end of the genome via homology to a transcriptional regulatory sequence (TRS, also known as an intergenic sequence), and requiring a discontinuous activity of the nascent minus-strand template and polymerase complex to acquire the leader (Sawicki and Sawicki, 1998). The outcome of transcription is the generation of a “nested set” of subgenomic negative-strand RNAs that all contain the antileader sequences that serve as templates for similar size subgenomic mRNAs. This transcriptional strategy exposes different genes as the 5′ ORF in different mRNAs, all of which also contain the 3′ sequence downstream of the gene, including the 3′ nontranslated region of the genome.
For MHV, genes 3, 5b, 6, and 7 encode S, E, M, and N, respectively. Genes 2, 4, and 5a are not required for replication in culture, and have been mutated to block expression, deleted, or substituted with noncoronavirus genes such as GFP (de Haan et al., 2002; Ortego et al., 2003; Sarma et al., 2002). Because all coronaviruses retain these genes in various combinations in the face of presumed pressure for genetic economy and apparent lack of functions in RNA synthesis, it is presumed that these genes serve roles in modification of host cells, pathogenesis, or interactions with the immune system. SCoV encodes a larger and more complex array of these genes than MHV or other coronaviruses, which may reflect its evolution in its original animal host (Ksiazek et al., 2003; Marra et al., 2003; Snijder et al., 2003; Thiel et al., 2003). In addition, the report of a deletion within one of the accessory genes in human isolates of SCoV suggests that this may be a gene involved in host range or adaptation for replication and transmission in humans (Guan et al., 2003).
Coronavirus Genetics
Until recently, the genetics of coronavirus replication and pathogenesis have largely been studied using natural variants, host range mutants, passaged virus, and mutagenized viruses selected for temperature sensitivity and specific phenotypes. Classical complementation of functions made it possible to define at least eight genetic groups for MHV, with most of the complementation groups localized to the replicase gene (Stalcup et al., 1998). Taking advantage of naturally high rates of homologous RNA-RNA recombination and of host range determinants in the S protein, the development of targeted recombination has allowed more defined and detailed studies of the accessory and structural genes of MHV, transmissible gastroenteritis virus (TGEV), and feline infectious peritonitis virus (FIPV) (Haijema et al., 2003; Kuo et al., 2000; Masters et al., 1994). Studies with natural variants and targeted recombination genetic studies have demonstrated that the S protein is the major determinant of host range, tropism, and pathogenesis; other genetic elements, possibly in the replicase, may influence these characteristics of different coronaviruses (Navas and Weiss, 2003). The capacity of coronaviruses to change host range, transmission, pathogenesis, and disease has been established in the laboratory using cell adaptation and virus passage (Baric et al., 1997, 1999; Chen and Baric, 1995, 1996), and has been demonstrated in nature by natural variants of MHV, TGEV, and bovine coronavirus (BCoV), as well as by studies using heterologous viruses such as canine coronavirus (CcoV) to immunize cats against FIPV (Enjuanes et al., 1995; Tresnan et al., 1996). Further, targeted recombination studies have confirmed the genetic flexibility of the coronavirus genome and the ability of coronaviruses to recover wild-type replication following deletions, mutations, substitutions, and gene order rearrangements in the structural and accessory genes (de Haan et al., 2002).
Challenges for genetic studies using natural variants and mutants, particularly in defining the precise changes responsible for altered phenotypes, has limited progress in genetic studies. Targeted recombination, while a robust system with powerful selection, has been limited to studies of the 3′ 10 kb of the MHV genome, and is limited to selection of viable recombinants. Recently, the establishment of “infectious clone” reverse genetic strategies for the coronaviruses TGEV (Transmissible Gastroenteritis Coronavirus), HCoV-229E, IBV, and MHV has made it theoretically possible to study the genetics of the entire genome and all of the structural, accessory, and replicase genes. Approaches to “infectious cloning” have included full-length cDNA clones of TGEV genome in bacterial artificial chromosomes (Gonzalez et al., 2001), recombinant vaccinia viruses containing full-length cDNA clones of HCoV-229E and IBV genomes (Casais et al., 2001; Thiel et al., 2001), and in vitro assembly strategies for TGEV, MHV, and most recently, SCoV (Yount, 2000; Yount et al., 2002, 2003).
The in vitro assembly approach was developed to overcome the challenge of full-length cDNA cloning of the TGEV and MHV genomes, which contained “toxic” regions in the replicase gene, resulting in unstable or toxic clones in E. coli (Yount, 2000; Yount et al., 2002). Subcloning of the regions required splitting the toxic domains into separate clones. The result of this strategy was the cloning of the MHV genome into seven fragments (A through G). To recover viable virus, the following strategy is pursued: (1) cloned cDNA fragments are excised from plasmid using class 2 restriction enzymes that remove the recognition site and leave overhanging genomic sequence; (2) excised fragments are ligated (assembled) in vitro; (3) transcription of full-length genomic RNA is driven in vitro using a T7 promoter on the 5′ fragment A; (4) full-length genome RNA is electroporated into competent cells that are then plated on a monolayer of naturally permissive cells; and (5) cells are monitored for cytopathic effect or plaques, and virus is recovered from plaques or media supernatant.
In vitro assembly has many advantages for genetic studies of such a large and complex genome RNA. First, genetic changes can be introduced and confirmed in stable small fragments without the need for a ~30kb genomic clone. Second, the cloned fragments make it possible to develop libraries of mutations that can rapidly be tested in different combinations. Furthermore, identification of putative second-site reversion mutations for deleterious introduced changes can be introduced with the original mutation to confirm their reversion potential. In combination with biochemical and cell imaging approaches, it also is possible to study highly defective or lethal mutations in electroporated cells, in order to define critical determinants of replication. The in vitro assembly approach has been used to introduce marker mutations that are silent for replication in culture (Yount et al., 2003). In addition, we have engineered mutations in the MHV replicase gene to define the requirements for polyprotein processing and to determine the role of specific replicase proteins in replication in culture and in pathogenesis in animals. Using this approach we have recovered viruses with mutations at polyprotein cleavage sites and proteinase catalytic residues, all of which have distinct phenotypes in protein processing, viral growth, and viral RNA synthesis (unpublished results). Thus, direct reverse genetic studies of the critical replicase gene functions can be performed using in vitro assembly of infectious clones.
Advances in SCoV Research
The rapid progress in the identification and characterization of SCoV as the etiologic agent of SARS was made possible by the fact that the virus grows well in culture, and by the foundational research in coronaviruses that has been supported by the National Institutes of Health, the Multiple Sclerosis Foundation, the U.S. Department of Agriculture, and other organizations over the past two decades. The application of knowledge concerning virus structure, genetics, receptor binding, virus entry, and viral pathogenesis has made it possible to target the spike protein for studies of SCoV replication, pathogenesis, and immune response (Xiao et al., 2003). The remarkably rapid identification of ACE 2 as a receptor for SARS has demonstrated the foundational importance of studies of other coronaviruses (Li et al., 2003). Similarly, understanding of replicase gene expression, processing, and predicted functions has identified possible targets for structure/function studies and possible therapeutic intervention. The studies of coronavirus proteinase activities, cleavage site, and structures were the basis for studies leading to the rapid determination of SCoV replicase polyprotein cleavage sites and 3CLpro crystal structure (Anand et al., 2003; Campanacci et al., 2003; Snijder et al., 2003; Thiel et al., 2003).
Application of Reverse Genetics to Studies of SCoV
Because of the potential for reemergence of SARS, it is important to move forward with research in diagnostics, vaccines, and therapeutics for SCoV. Experience with the development and use of reverse genetics to study other coronaviruses resulted in establishment of reverse genetics for SCoV within months of the onset of the worldwide epidemic (Yount et al., 2003). How should the understanding of other coronaviruses, the rapid advances in research with SCoV, and the development of reverse genetics for SCoV be harnesssed to achieve these goals and attack these critical questions in SCoV replication, pathogenesis, and disease? Certainly, the use of SCoV reverse genetics, along with robust tissue culture systems and emerging animal models, creates the potential to rapidly answer questions concerning: (1) determinants of virus growth in culture; (2) potential mechanisms of transpecies adaptation; (3) sensitivity to and escape from biochemical and immune interference with replication; (4) determinants of virulence and pathogenesis; (5) mechanisms of genome recombination and mutation; (6) functions of and requirements for replicase, structural, and accessory proteins; and (7) development of stably attenuated viruses for use as seed stocks for inactivated vaccine or testing as live-attenuated vaccines.
How then should these critical issues be investigated while recognizing the potential of SCoV to cause severe disease, as well as the potential for rapid spread? First, there is significant experience with other coronaviruses in attenuation of virus replication and pathogenesis, both using virus passage and by direct engineering of changes. Although coronavirus genome organization, proteins, and replication appear more tolerant of changes then previously thought, all changes of gene order, gene deletion, insertion, or mutagenesis so far reported have led to viruses impaired in replication, pathogenesis, or both. Many of the attenuating changes in MHV and other coronaviruses are conserved in SCoV and thus could be tested for likely attenuation in SCoV culture and animal models. Second, where there is clear conservation of sequences, motifs, proteins, or putative functions between SCoV and model viruses such as MHV, new or untested changes might be most rapidly analyzed under BSL2 conditions in those model viruses, and then directly applied to SARS once their phenotypes are determined. Third, all work with SCoV will be performed only under BSL3 conditions. This would also apply to chimeric viruses, whether engineered by introduction into the SCoV background, or by introducing SCoV proteins or sequences with known or predicted pathogenic consequences into other coronavirus backgrounds. Finally, it is important to develop strains of SCoV that are attenuated and stabilized against reversion and recombination, to be used as the basis for studies of other replication and pathogenesis determinants and construction of virus chimeras. Such attenuated variants would provide additional safeguards while allowing application of powerful genetic tools to the study of SCoV emergence, biology, disease, treatment, and prevention. Overall, newly invigorated programs in other human and animal coronaviruses, combined with the new research in SCoV, will shed important new light on this important virus family and perhaps lead to better understanding of the potential for resurgence of SCoV or the emergence of other coronaviruses into human populations."""

In [ ]:
def display_entities(model,document):
    """ A function that returns a tuple of displacy image of named or unnamed word entities and
        a set of unique entities recognized based on scispacy model in use
        Args: 
            model: A pretrained model from spaCy or ScispaCy
            document: text data to be analysed"""
    nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = set([(X.text, X.label_) for X in doc.ents])
    return  displacy_image, entity_and_label

In [ ]:
bc5dr_entities = display_entities(en_ner_bc5cdr_md,sample_text)

In [ ]:
bc5dr_entities_dataframe = pd.DataFrame(bc5dr_entities[1],columns=['Entity','Label'])  #save returned values of entities and their labels in a pandas dataframe
bc5dr_entities_dataframe['Ner_model'] = 'bc5dr'  #include a column with constant value of NER model
bc5dr_entities_dataframe

In [ ]:
bionlp13cg_entities = display_entities(en_ner_bionlp13cg_md,sample_text)

In [ ]:
bionlp13cg_entities_dataframe = pd.DataFrame(bionlp13cg_entities[1],columns=['Entity','Label']) #save returned values of entities and their labels in a pandas dataframe
bionlp13cg_entities_dataframe['Ner_model'] = 'bionlp13cg'  #include a column with constant value of NER model
bionlp13cg_entities_dataframe

In [ ]:
craft_entities = display_entities(en_ner_craft_md,sample_text)

In [ ]:
craft_entities_dataframe = pd.DataFrame(craft_entities[1],columns=['Entity','Label'])  #save returned values of entities and their labels in a pandas dataframe
craft_entities_dataframe['Ner_model'] = 'craft' #include a column with constant value of NER model
craft_entities_dataframe

In [ ]:
jnlpba_entities = display_entities(en_ner_jnlpba_md,sample_text)

In [ ]:
jnlpa_entities_dataframe = pd.DataFrame(jnlpba_entities[1],columns=['Entity','Label']) #save returned values of entities and their labels in a pandas dataframe
jnlpa_entities_dataframe['Ner_model'] = 'jnlpa' # include a column with constant value of NER model
jnlpa_entities_dataframe

In [ ]:
entities_and_label_from_4_NER_model_dataframe = pd.concat([bc5dr_entities_dataframe,bionlp13cg_entities_dataframe,craft_entities_dataframe,jnlpa_entities_dataframe])
#Concatenate all pandas dataframe into one.
entities_and_label_from_4_NER_model_dataframe.to_csv('entities_and_label_from_4_scispacy_NER_models.csv', index=False) #Save dataframe to csv
entities_and_label_from_4_NER_model_dataframe.info()

In [ ]:
#Load pre extracted dataset into pandas
entities_and_label_from_4_NER_model_dataframe = pd.read_csv('/content/entities_and_label_from_4_scispacy_NER_models.csv')

In [ ]:
#pd.options.display.max_colwidth = 1000      #increase display width of the pandas dataframe
entities_and_label_from_4_NER_model_dataframe.head() 

In [ ]:
def entity_linker(linker_name,document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    nlp = en_core_sci_sm.load()
    nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

###Test the entity_linker function with the 4 sciSpacy knowledge bases "umls","mesh","go","hpo". The function will return 2 entities and their scores as it relates to the Knowledge base. As at the time of this writing the "rxnorm" knowledge base returned KeyError, I will investigate further on why this is happening

In [ ]:
test1 = 'fever'

In [ ]:
entity_linker('umls',test1)

In [ ]:
entity_linker('go',test1)

In [ ]:
entity_linker('hpo',test1)

In [ ]:
entity_linker('mesh',test1)

In [ ]:
entity_linker('rxnorm',test1)

In [ ]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='umls')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_core_sci_sm.load()
nlp.add_pipe(linker)
def umls_entity_linker(document):
    """ A function that accepts and document and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [ ]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='mesh')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_core_sci_sm.load()
nlp.add_pipe(linker)
def mesh_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [ ]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='go')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_core_sci_sm.load()
nlp.add_pipe(linker)
def go_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [ ]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='hpo')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_core_sci_sm.load()
nlp.add_pipe(linker)
def hpo_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [ ]:
entities_and_label_from_4_NER_model_dataframe['umls_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : umls_entity_linker(x))

In [ ]:
entities_and_label_from_4_NER_model_dataframe['mesh_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : mesh_entity_linker(x))

In [ ]:
entities_and_label_from_4_NER_model_dataframe['go_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : go_entity_linker(x))

In [ ]:
entities_and_label_from_4_NER_model_dataframe['hpo_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : hpo_entity_linker(x))

In [ ]:
pd.options.display.max_colwidth = 1000      #increase display width of the pandas dataframe
entities_and_label_from_4_NER_model_dataframe.head()